In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import sys,bs4,re,time,requests,pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#url = "https://www.contractsfinder.service.gov.uk/notice/210345e7-e8b1-43b3-a932-e01d90b4eaf6?origin=SearchResults&p=1"
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [3]:
titles = []
hrefs = []
thumbnails = []

range_count = 10
for x in range(0,range_count):
    try:
        api = f"https://go.buy.mi.com/global/page/discovery?from=mobile&page_num={x}&show_type=newsroom&cate_id=0"
        req = requests.get(api)
        data = req.json()['data']['page_data']
        for i in data:
            title = i['assembly_info'][0]['title'] 
            href = i['assembly_info'][0]['go_to_url'] 
            thumb = i['assembly_info'][0]['image_url'] 
            titles.append(title)
            hrefs.append(href)
            thumbnails.append(thumb)
    except:
        pass
    
    print(str(range_count-(x+1)) + " pages remaining..",end="\r")

In [4]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [5]:
dates,articles = [],[]
range_count = len(href)

try:
    for number,link in enumerate(hrefs[:range_count]):

        driver.maximize_window()

        driver.get(link)

        date = driver.find_element_by_xpath(f'''//*[@id="root"]/div/main/div/section/div/div[2]/span[1]''').text
        dates.append(date)

        article = driver.find_element_by_xpath(f'''//*[@id="root"]/div/main/div[2]/section''').text
        articles.append(article)
        
        print(str(range_count-(number+1)) + " pages remaining..",end="\r")
except:
    pass

In [6]:
#zippling the list so that it'll form the tuple
zipped = list(zip(dates,titles,articles,hrefs,thumbnails))
zipped[:3]

[('25/04/2022',
  'Redmi 10A: Exceptional Value in International Markets',
  'Redmi 10A: Exceptional Value in International Markets\n25/04/20221295 Views\nBeijing, China, April 25th 2022 – Xiaomi today announced Redmi 10A, the latest member of Redmi 10 series family, dedicated to bringing consumers a well-round user experience at an affordable price.\nRedmi 10A offers a large 6.53" HD+ display to provide a better experience for watching videos as well as contents you enjoy. The 3D-curved back produces a natural feel to Redmi 10A that other entry-level smartphones can’t match. The device also features a rear fingerprint sensor to make it more convenient to unlock.\nComes with a 13MP main camera for capturing and sharing your life, as well as a 2MP depth camera that helps to achieve a natural blur effect in the background when you take portraits, Redmi 10A is comfortably capable of recording your memorable moments of daily life.\nPowering up Redmi 10A is a MediaTek Helio G25 processor, w

In [7]:
#creating dataframe 
temp_df = pd.DataFrame(zipped,columns=['date','title','article','url','thumbnail'])
temp_df

,date,title,article,url,thumbnail
0,25/04/2022,Redmi 10A: Exceptional Value in International ...,Redmi 10A: Exceptional Value in International ...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/870_operatorx_operat...
1,06/04/2022,Xiaomi Announces Xiaomi Fan Festival 2022 With...,Xiaomi Announces Xiaomi Fan Festival 2022 With...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/211_operatorx_operat...
2,31/03/2022,The Earthquake Early Warning Function on Xiaom...,The Earthquake Early Warning Function on Xiaom...,http://mobile.mi.com/global/discover/article?i...,https://i01.appmifile.com/webfile/globalimg/PR...
3,29/03/2022,Xiaomi Debuts Latest 5G Additions to Redmi Not...,Xiaomi Debuts Latest 5G Additions to Redmi Not...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/590_operatorx_operat...
4,22/03/2022,Xiaomi Reports 70% YoY Growth in Adjusted Net ...,Xiaomi Reports 70% YoY Growth in Adjusted Net ...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/69_operatorx_operato...
5,19/03/2022,Introducing Redmi 10C – Great Specs at Entry-...,Introducing Redmi 10C – Great Specs at Entry-l...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/330_operatorx_operat...
6,17/03/2022,Redmi K50 Series and a host of AIoT products a...,Redmi K50 Series and a host of AIoT products a...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/80_operatorx_operato...
7,15/03/2022,Xiaomi Levels up Wearable Experience with All-...,Xiaomi Levels up Wearable Experience with All-...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/49_operatorx_operato...
8,,Xiaomi 12 Series Redefines Flagship Category,0\n0,http://mobile.mi.com/global/discover/article?i...,https://i01.appmifile.com/webfile/globalimg/PR...
9,03/03/2022,Xiaomi Contributes to Advancing 5G Standards T...,Xiaomi Contributes to Advancing 5G Standards T...,http://mobile.mi.com/global/discover/article?i...,https://i02.appmifile.com/796_operatorx_operat...


In [8]:
#to csv
temp_df.to_csv('mi news.csv')

#to json
temp_df.to_json('mi news.json')